<a href="https://colab.research.google.com/github/41371116h/PL-Repo./blob/main/hw1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import gradio as gr
import pandas as pd
import datetime
import gspread
from google.colab import auth
from google.auth import default

# ===== Google Sheets 設定 =====
SHEET_URL = "https://docs.google.com/spreadsheets/d/1GScHTHISiioV89XO5twgGl-IpaYG-0nMwhLRTizSpNI/edit?pli=1&gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

# ===== Google 授權 =====
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url(SHEET_URL)
worksheet = gsheets.worksheet(WORKSHEET_NAME)

# 請款人選單
PEOPLE_LIST = ["黃惠榆", "謝宜君", "白曉辰", "顧晏", "江燮", "謝云森"]

# ===== 寫入函數 =====
def add_data(date_str, time_str, item, price, number, people, reimbursement):
    try:
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        datetime.datetime.strptime(time_str, '%H:%M')

        price = float(price)
        number = float(number)
        expense = price * number

        new_row = [date_str, time_str, item, price, number, expense, people, reimbursement]
        worksheet.append_row(new_row, value_input_option='USER_ENTERED')

        return "✅ 資料已成功寫入 Google 工作表！"
    except ValueError:
        return "❌ 日期或時間格式錯誤，請使用 YYYY-MM-DD 與 HH:MM 格式"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"

# ===== 統計未請款金額 =====
def calculate_unclaimed():
    try:
        all_data = worksheet.get_all_values()
        if not all_data or len(all_data) < 2:
            return "目前沒有資料。"

        df = pd.DataFrame(all_data[1:], columns=all_data[0])
        df["支出"] = pd.to_numeric(df["支出"], errors="coerce")
        unclaimed = df[df["是否核銷"] == "否"]
        if unclaimed.empty:
            return "目前沒有未請款金額。"

        grouped = unclaimed.groupby("請款人")["支出"].sum()
        result = "\n".join([f"{person}: {amount}" for person, amount in grouped.items()])
        return f"📊 各請款人未請款金額：\n{result}"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"

# ===== Gradio Tabs 介面 =====
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Google 試算表系統")

    with gr.Tabs():
        # --- 分頁1：資料輸入 ---
        with gr.Tab("資料輸入"):
            with gr.Row():
                date_in = gr.Textbox(label="日期 (YYYY-MM-DD)")
                time_in = gr.Textbox(label="時間 (HH:MM)")
            item_in = gr.Textbox(label="品項")
            with gr.Row():
                price_in = gr.Number(label="單價")
                number_in = gr.Number(label="數量")
            people_in = gr.Dropdown(choices=PEOPLE_LIST, label="請款人", value=PEOPLE_LIST[0])
            reimbursement_in = gr.Dropdown(choices=["是", "否"], label="是否核銷", value="否")

            submit_output = gr.Textbox(label="結果", lines=6)
            submit_btn = gr.Button("寫入 Google 工作表")
            submit_btn.click(
                fn=add_data,
                inputs=[date_in, time_in, item_in, price_in, number_in, people_in, reimbursement_in],
                outputs=submit_output
            )

        # --- 分頁2：未請款金額統計 ---
        with gr.Tab("未請款金額統計"):
            stats_output = gr.Textbox(label="未請款統計結果", lines=10)
            stats_btn = gr.Button("更新未請款金額")
            stats_btn.click(fn=calculate_unclaimed, inputs=None, outputs=stats_output)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a7db32606b2dfb241.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
